# MemTorch Tutorial

## 1. Training and benchmarking a DNN using CIFAR-10

The VGG-16 DNN architecture is trained and tested using the CIFAR-10 data set. The CIFAR-10 data set consists of 60,000 32x32 color images in 10 classes, with 6,000 images per class. There are 50,000 training images and 10,000 test images. The network is trained for 50 epochs with a batch size, $\Im=256$. The initial learning rate is $\eta = 1e-2$, which is decayed by an order of magnitude every 20 training epochs. Adam is used to optimize network parameters and Cross Entropy (CE) is used to determine network losses. *memtorch.utils.LoadCIFAR10* is used to load the CIFAR-10 training and test sets. After each epoch the model is bench-marked using the CIFAR-10 test set. The model that achieves the highest test set accuracy is saved as *trained_model.pt*.

In [ ]:
import torch
from torch.autograd import Variable
import memtorch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from memtorch.utils import LoadCIFAR10
import numpy as np


class Net(nn.Module):
    def __init__(self, inflation_ratio=1):
        super(Net, self).__init__()
        self.conv0 = nn.Conv2d(in_channels=3, out_channels=128*inflation_ratio, kernel_size=3, stride=1, padding=1)
        self.bn0 = nn.BatchNorm2d(num_features=128*inflation_ratio)
        self.act0 = nn.ReLU()
        self.conv1 = nn.Conv2d(in_channels=128*inflation_ratio, out_channels=128*inflation_ratio, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=128*inflation_ratio)
        self.act1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=128*inflation_ratio, out_channels=256*inflation_ratio, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=256*inflation_ratio)
        self.act2 = nn.ReLU()
        self.conv3 = nn.Conv2d(in_channels=256*inflation_ratio, out_channels=256*inflation_ratio, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=256*inflation_ratio)
        self.act3 = nn.ReLU()
        self.conv4 = nn.Conv2d(in_channels=256*inflation_ratio, out_channels=512*inflation_ratio, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(num_features=512*inflation_ratio)
        self.act4 = nn.ReLU()
        self.conv5 = nn.Conv2d(in_channels=512*inflation_ratio, out_channels=512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(num_features=512)
        self.act5 = nn.ReLU()
        self.fc6 = nn.Linear(in_features=512*4*4, out_features=1024)
        self.bn6 = nn.BatchNorm1d(num_features=1024)
        self.act6 = nn.ReLU()
        self.fc7 = nn.Linear(in_features=1024, out_features=1024)
        self.bn7 = nn.BatchNorm1d(num_features=1024)
        self.act7 = nn.ReLU()
        self.fc8 = nn.Linear(in_features=1024, out_features=10)

    def forward(self, input):
        x = self.act0(self.bn0(self.conv0(input)))
        x = self.act1(self.bn1(F.max_pool2d(self.conv1(x), 2)))
        x = self.act2(self.bn2(self.conv2(x)))
        x = self.act3(self.bn3(F.max_pool2d(self.conv3(x), 2)))
        x = self.act4(self.bn4(self.conv4(x)))
        x = self.act5(self.bn5(F.max_pool2d(self.conv5(x), 2)))
        x = x.view(x.size(0), -1)
        x = self.act6(self.bn6(self.fc6(x)))
        x = self.act7(self.bn7(self.fc7(x)))
        return self.fc8(x)

def test(model, test_loader):
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):        
        output = model(data.to(device))
        pred = output.data.max(1)[1]
        correct += pred.eq(target.to(device).data.view_as(pred)).cpu().sum()

    return 100. * float(correct) / float(len(test_loader.dataset))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 50
train_loader, validation_loader, test_loader = LoadCIFAR10(batch_size=256, validation=False)
model = Net().to(device)
if device == 'cuda':
    model = torch.nn.DataParallel(model)

criterion = nn.CrossEntropyLoss()
learning_rate = 1e-2
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
best_accuracy = 0
for epoch in range(0, epochs):
    print('Epoch: [%d]\t\t' % (epoch + 1), end='')
    if epoch % 20 == 0:
        learning_rate = learning_rate * 0.1
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate

    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        loss = criterion(output, target.to(device))
        loss.backward()
        optimizer.step()

    accuracy = test(model, test_loader)
    print('%2.2f%%' % accuracy)
    if accuracy > best_accuracy:
        torch.save(model.state_dict(), 'trained_model.pt')
        best_accuracy = accuracy

## 2. Conversion of a DNN to a MDNN 

We use MemTorch to demonstrate the conversion of a DNN to MDNN. A memristive device model is defined and characterized below, which is used to replace all *torch.nn.Linear* layers within the DNN, trained in Step 1, with equivalent crossbar architectures using *memtorch.mn.Module.patch_model*.

In [ ]:
reference_memristor = memtorch.bh.memristor.VTEAM
reference_memristor_params = {'time_series_resolution': 1e-10}
memristor = reference_memristor(**reference_memristor_params)
memristor.plot_hysteresis_loop()

*memtorch.bh.map.Parameter.naive_map* is used to convert the weights within all *torch.nn.Linear* layers to equivalent conductance values, to be programmed to the two memristive devices used to represent each weight (positive and negative) using Eq. (13). 

*transistor* is *True*, so a 1T1R arrangement is simulated. *programming_routine* is set to *None* to skip device-level simulation of the programming routine. We note if *transistor* is *False* *programming_routine* must not be *None*. In which case, device-level simulation is performed for each device using *memtorch.bh.crossbar.gen_programming_signal* and *memtorch.bh.memristor.Memristor.simulate*, which uses finite differences to model internal device dynamics. As *scheme* is not defined, a double-column parameter representation scheme is adopted.

All patched *torch.nn.Linear* layers are tuned using linear regression. A randomly generated tensor of size (8, *self.in_features*) is propagated through each memristive layer and each legacy layer (accessible using *layer.forward_legacy*). *sklearn.linear_model.LinearRegression* is used to determine the coefficient and intercept between the linear relationship of each set of outputs, which is used to define the *transform_output* lamdba function, that maps the output of each layer to their equivalent representations.


In [ ]:
import copy
from memtorch.mn.Module import patch_model
from memtorch.map.Parameter import naive_map
from memtorch.bh.crossbar.Program import naive_program


model = Net()
if torch.cuda.is_available():
  model = torch.nn.DataParallel(model)

model.load_state_dict(torch.load('trained_model.pt'), strict=False)
patched_model = patch_model(copy.deepcopy(model),
                          memristor_model=reference_memristor,
                          memristor_model_params=reference_memristor_params,
                          module_parameters_to_patch=[torch.nn.Linear],
                          mapping_routine=naive_map,
                          transistor=True,
                          programming_routine=None)


In [ ]:
patched_model.tune_()

In [ ]:
print(test(patched_model, test_loader))

## 3. Modeling non-ideal device characteristics

We use a simple prototype model to demonstrate modeling non-ideal device characteristics.

In [ ]:
from memtorch.mn.Module import patch_model
import copy
from memtorch.map.Parameter import naive_map

class Model(torch.nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.convolutional_layer = torch.nn.Conv2d(in_channels=3, out_channels=1, kernel_size=5)
        self.linear_layer = torch.nn.Linear(in_features=16, out_features=4)

    def forward(self, input):
        x = self.convolutional_layer(input)
        x = x.view(x.size(0), -1)
        return self.linear_layer(x)
    
torch.manual_seed(0)
model = Model()
if torch.cuda.is_available():
  model = torch.nn.DataParallel(model)

reference_memristor_params = {'time_series_resolution': 1e-10, 'r_off': 200, 'r_on': 100}
patched_model = patch_model(copy.deepcopy(model),
                          memristor_model=reference_memristor,
                          memristor_model_params=reference_memristor_params,
                          module_parameters_to_patch=[torch.nn.Linear, torch.nn.Conv2d],
                          mapping_routine=naive_map,
                          transistor=True,
                          programming_routine=None)

Device-device variability is introduced to the VTEAM reference memristor model using *memtorch.bh.StochasticParameter*, by sampling $R_{\text{OFF}}$ for each device from a normal distribution with $\sigma = 20$ and $x = 200$, and  $R_{\text{ON}}$ for each device from a normal distribution with $\sigma = 10$ and $x = 100$. Using *np.vectorize*, the $R_{\text{OFF}}$ and $R_{\text{ON}}$ values for each memristive device are compared.

In [ ]:
from memtorch.mn.Module import patch_model
import copy
from memtorch.map.Parameter import naive_map
from memtorch.bh.crossbar.Program import naive_program
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

reference_memristor_params = {'time_series_resolution': 1e-10, 
                              'r_off': memtorch.bh.StochasticParameter(200, std=20, min=2),
                              'r_on': memtorch.bh.StochasticParameter(100, std=10, min=1)}

patched_model_ = patch_model(copy.deepcopy(model),
                          memristor_model=reference_memristor,
                          memristor_model_params=reference_memristor_params,
                          module_parameters_to_patch=[torch.nn.Linear, torch.nn.Conv2d],
                          mapping_routine=naive_map,
                          transistor=True,
                          programming_routine=None)

A = torch.Tensor(np.vectorize(lambda x: x.r_off)(patched_model_.linear_layer.crossbars[0].devices))
B = torch.Tensor(np.vectorize(lambda x: x.r_on)(patched_model_.linear_layer.crossbars[0].devices))
C = torch.cat((A, B), 0)


plt.subplot(2, 1, 1)
plt.imshow(A.transpose(0, 1), interpolation='nearest', aspect=1, vmin=C.min(), vmax=C.max(), cmap='seismic')
plt.xticks([])
plt.yticks([])
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 0')
divider = make_axes_locatable(plt.gca())
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(cax=cax)

plt.subplot(2, 1, 2)
plt.imshow(B.transpose(0, 1), interpolation='nearest', aspect=1, vmin=C.min(), vmax=C.max(), cmap='seismic')
plt.xticks([])
plt.yticks([])
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 0')
divider = make_axes_locatable(plt.gca())
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(cax=cax)
plt.savefig('var.svg')
plt.show()

We model a number (5) of finite discrete conductance states using $memtorch.bh.nonideality.NonIdeality.FiniteConductanceStates$. The conductance levels within the positive crossbar were compared before and after a finite discrete conductance states are modeled.

In [ ]:
from memtorch.bh.nonideality.NonIdeality import apply_nonidealities

A = 1 / patched_model.linear_layer.crossbars[0].conductance_matrix
model = apply_nonidealities(copy.deepcopy(patched_model),
                                  non_idealities=[memtorch.bh.nonideality.NonIdeality.FiniteConductanceStates],
                                  conductance_states = 5)
B = 1 / model.linear_layer.crossbars[0].conductance_matrix
C = torch.cat((A, B), 0)

plt.subplot(2, 1, 1)
plt.imshow(A.transpose(0, 1), interpolation='nearest', aspect=1, vmin=C.min(), vmax=C.max(), cmap='seismic')
plt.xticks([])
plt.yticks([])
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 0')
divider = make_axes_locatable(plt.gca())
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(cax=cax)

plt.subplot(2, 1, 2)
plt.imshow(B.transpose(0, 1), interpolation='nearest', aspect=1, vmin=C.min(), vmax=C.max(), cmap='seismic')
plt.xticks([])
plt.yticks([])
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 0')
divider = make_axes_locatable(plt.gca())
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(cax=cax)
plt.savefig('finite.svg')
plt.show()

We model device failure using $memtorch.bh.nonideality.NonIdeality.DeviceFaults$. The conductance levels within the positive crossbar are once again compared, before and after  50\% of devices are stuck at $R_{\text{LRS}}$ and 50\% of devices are stuck at $R_{\text{HRS}}$.

In [ ]:
from memtorch.bh.nonideality.NonIdeality import apply_nonidealities

A = 1 / patched_model.linear_layer.crossbars[0].conductance_matrix
model = apply_nonidealities(copy.deepcopy(patched_model),
                                  non_idealities=[memtorch.bh.nonideality.NonIdeality.DeviceFaults],
                                  lrs_proportion=0.5,
                                  hrs_proportion=0.5,
                                  electroform_proportion=0)
B = 1 / model.linear_layer.crossbars[0].conductance_matrix
C = torch.cat((A, B), 0)

plt.subplot(2, 1, 1)
plt.imshow(A.transpose(0, 1), interpolation='nearest', aspect=1, vmin=C.min(), vmax=C.max(), cmap='seismic')
plt.xticks([])
plt.yticks([])
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 0')
divider = make_axes_locatable(plt.gca())
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(cax=cax)

plt.subplot(2, 1, 2)
plt.imshow(B.transpose(0, 1), interpolation='nearest', aspect=1, vmin=C.min(), vmax=C.max(), cmap='seismic')
plt.xticks([])
plt.yticks([])
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 0')
divider = make_axes_locatable(plt.gca())
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(cax=cax)
plt.savefig('fault.svg')
plt.show()

We model non-linear I/V characteristics using $memtorch.bh.nonideality.NonIdeality.NonLinear$ during inference. The output of the single *torch.nn.Linear* layer are compared when devices were simulated during inference with linear and non-linear I/V characteristics. Non-linear I/V characteristics were determined by applying a half-voltage sweep, using a sinusoidal cosine voltage signal with a duration of 5ns, amplitude of 1V, and a frequency of 50 MHz to each device. 

In [ ]:
from memtorch.bh.nonideality.NonIdeality import apply_nonidealities
from sklearn.metrics.pairwise import cosine_similarity

A = torch.tensor(np.zeros((100, 4)))
B = torch.tensor(np.zeros((100, 4)))
for i in range(100):
  input = torch.zeros((1,3,8,8)).uniform_(-1, 1)
  A[i, :] = patched_model(input)
  model = apply_nonidealities(copy.deepcopy(patched_model),
                              non_idealities=[memtorch.bh.nonideality.NonIdeality.NonLinear],
                              sweep_duration=5e-9,
                              sweep_voltage_signal_amplitude=1,
                              sweep_voltage_signal_frequency=50e6)
  B[i, :] = model(input)

print(cosine_similarity([A.view(-1).numpy()], [B.view(-1).numpy()]))